In [2]:
import pandas as pd
import glob
from googletrans import Translator
from google_trans_new import google_translator  
import emoji
import time
import re
from langdetect import detect
import demoji
import fasttext

In [3]:
directory = 'targeted'
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)  # Show all rows
#pd.set_option('display.max_columns', None)

In [4]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001F680-\U0001F6FF"  # CJK Unified Ideographs Extension B
                           u"\U0001F600-\U0001F64F"  # CJK Unified Ideographs
                           u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols and Pictographs"
                           u"\U0001F000-\U0001F02F"  # Mahjong Tiles
                           u"\U0001F004-\U0001F0CF"  # Domino Tiles
                           u"\U0001F110-\U0001F251"  # Playing Cards
                           u"\U0001F600-\U0001F64F"  # Emoticons (faces)
                           u"\U0001F910-\U0001F91F"  # Emojis (animal faces)
                           u"\U0001F920-\U0001F927"  # Emojis (human faces)
                           u"\U0001F930-\U0001F939"  # Emojis (clothing)
                           u"\U0001F950-\U0001F95F"  # Emojis (food)
                           u"\U0001F980-\U0001F991"  # Emojis (objects)
                           u"\U0001F9C0-\U0001F9C0"  # Emojis (symbols)
                           "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', text)



In [5]:
t = Translator()

translations = []
test_file = glob.glob(directory + "/*.xlsx")

test_list = []
for file in test_file:
    df = pd.read_excel(file, sheet_name='Tweets')
    test_list.append(df)


# Concatenate the DataFrames in test_list into a single DataFrame
combined_df = pd.concat(test_list, axis=0, ignore_index=True)
combined_df = combined_df.drop(['Binders', 'Tweet value', 'Likes count', 'Retweet count'], axis = 1)
combined_df['Tweet'] = combined_df['Tweet'].str.replace("’", "'")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("‘", "'")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("“", "\"")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("”", "\"")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("‼️", "!!")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("…", "...")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("⁃", "-")
combined_df['Tweet'] = combined_df['Tweet'].str.replace("☹️", "")
combined_df['Tweet'] = combined_df['Tweet'].apply(remove_emojis)



language_model = fasttext.load_model('lid.176.bin') 

test_file = glob.glob(directory + "/*.xlsx")

test_list = []
for file in test_file:
    df = pd.read_excel(file, sheet_name='Tweets')
    test_list.append(df)

combined_df = pd.concat(test_list, axis=0, ignore_index=True)
combined_df = combined_df.drop(['Binders', 'Tweet value', 'Likes count', 'Retweet count'], axis=1)

#initalize already to English
combined_df['Is_English'] = '1'

for index, row in combined_df.iterrows():
    text = row['Tweet']
    text = text.replace('\n', ' ')
    try:
        prediction = language_model.predict(text)
        detected_language = prediction[0][0].replace("__label__", "")
        
        # Define a confidence threshold to consider a language as detected
        confidence_threshold = 0.8 
        if prediction[1][0] >= confidence_threshold:
            if detected_language == 'en':
                combined_df.at[index, 'Is_English'] = '1'  # English
            else:
                combined_df.at[index, 'Is_English'] = '0'  # Non-English
    except Exception as e:
        print(f"Error detecting language: {e}")

mask = combined_df['Is_English'].str.contains('0', case=False)


# Use the mask to filter rows that match the condition
filtered_rows = combined_df[mask]

filtered_rows.to_csv('filtered.csv')

df = pd.read_csv('filtered.csv')
# Extract the 'Tweet' column
texts = df['Tweet'].tolist()

batch_size = 10  

translations = []

for i in range(0, len(texts), batch_size):
    text_batch = texts[i:i + batch_size]
    translated_batch = []

    for text in text_batch:
        try:
            translated_text = t.translate(text, dest='en').text
            if translated_text is not None:
                translated_batch.append(translated_text)
            else:
                translated_batch.append("")
            time.sleep(1)
        except Exception as e:
            print(f"Error translating: {e}")
            translated_batch.append("Wrong") # Placeholder to determine which non-English text is causing issues
    translations.extend(translated_batch)

df['Translated'] = translations




Error translating: sequence item 5: expected str instance, NoneType found
Error translating: sequence item 3: expected str instance, NoneType found
Error translating: sequence item 5: expected str instance, NoneType found
Error translating: sequence item 3: expected str instance, NoneType found
Error translating: sequence item 9: expected str instance, NoneType found
Error translating: sequence item 6: expected str instance, NoneType found
Error translating: sequence item 5: expected str instance, NoneType found
Error translating: sequence item 2: expected str instance, NoneType found
Error translating: sequence item 3: expected str instance, NoneType found
Error translating: sequence item 3: expected str instance, NoneType found
Error translating: sequence item 3: expected str instance, NoneType found


In [22]:
# Some texts do not end up translating to English
df2 = df[df['Translated'].str.contains("Wrong")]
df2['Tweet'] = df2['Tweet'].str.replace("☹️", "")
df2.loc[:, 'Tweet'] = df2['Tweet'].apply(remove_emojis) #Emojis not removed for some reason
for index, row in df2.iterrows():
    text = row['Tweet']
    try:
        translated = t.translate(text, dest='en')  
        if translated:
            translated_text = translated.text
            df2.loc[index, 'Translated'] = translated_text
        else:
            df2.loc[index, 'Translated'] = "Translation Failed"
    except Exception as e:
        print(f"Error translating: {e}")
        df2.loc[index, 'Translated'] = "Translation Error"


/var/folders/4_/rf44p2zs3ld9skyp7yxhcm8r0000gn/T/ipykernel_14977/1047725229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Tweet'] = df2['Tweet'].str.replace("☹️", "")
/Users/BrianPark/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/BrianPark/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [23]:
df2

,Unnamed: 0,User,Username,UTC Date,Tweet,Permalink,Is_English,Translated
545,151192,piyari Ben Solanki Mamta Ben Solanki,@BenPyar,2022-08-06 11:11:47,"@Swamy39 @RupaliDubeyDl @asharamjibapu_ @PMOIndia Sanatan Warrior\nहोना कोई अपराध तो नही जो फेक केस में\nSant Shri Asharamji Bapu को उम्रकैद दे दी गई, उन्हें जेल हुई इसकी वजह है भ्रष्ट राजनेता जो चाहते ही नही हिंदुत्व जीवित रहे वरना अब तक निर्दोष संत को जेलमुक्त न करवा देते। \nIntentionally Targeted\n#भारत_के_संत https://t.co/gVLbGXgs5W",https://www.twitter.com/user/status/1555874269883551745,0,"@Swamy39 @Ruppalidubeydl @asharamajibapu_ @pmoindia sanatan warrior\nIt is not a crime that in a fake case\nSant Shri Asharamji Bapu was given life imprisonment, he was jailed because of this, the corrupt politicians who do not want Hindutva survive, otherwise they would not have made the innocent saint free till now.\nInternally targeted\n#Bharat_K_Sant https://t.co/GVLBGXGS5W"
614,153726,Mahesh chuglani,@Maheshchuglani5,2022-07-21 02:34:48,Sanatan Warrior\nSant Shri Asharamji Bapu ने भारतीयों को स्वधर्म रक्षण हेतु जागरूक किया धर्मान्तरित लोगों की घरवापसी करवाई तो Planned Conspiracy रचाई गई संत को जेल भिजवाया गया। \nIntentionally Targeted\n#भारत_के_संत https://t.co/T4EwsxxO37,https://www.twitter.com/user/status/1549945963489333248,0,"Sanatan warrior\nSant Shri Asharamji Bapu made Indians aware of self -righteousness and got the converted people's homes, then the saint, who was planned conspiracy, was sent to jail.\nInternally targeted\n#Bharat_K_Sant https://t.co/t4sxxo37"
622,153737,D G,@deepakgarg55555,2022-07-21 02:33:44,"@drybiswal @YssSpeaks Sanatan Warrior\nहोना कोई अपराध तो नही जो फेक केस में\nSant Shri Asharamji Bapu को उम्रकैद दे दी गई, उन्हें जेल हुई इसकी वजह है भ्रष्ट राजनेता जो चाहते ही नही हिंदुत्व जीवित रहे वरना अब तक निर्दोष संत को जेलमुक्त न करवा देते। \nIntentionally Targeted\n#भारत_के_संत https://t.co/SkOGx9hFin",https://www.twitter.com/user/status/1549945694470807552,0,"@Drybiswal @YSSSPEAKS Sanatan Warrior\nIt is not a crime that in a fake case\nSant Shri Asharamji Bapu was given life imprisonment, he was jailed because of this, the corrupt politicians who do not want Hindutva survive, otherwise they would not have made the innocent saint free till now.\nInternally targeted\n#Bharat_K_Sant https://t.co/skogx9hfin"
673,158124,Kshipra K,@k_kshipra,2022-06-18 11:22:03,"@YssSpeaks #PersecutionOfHindus by Serial Slanderers आश्चर्य की बात है कि बॉलीवुड के हर मूवी, हर सीरियल में हिंदू देवी देवताओं की मजाक बनाई जाती है! और फिर भी हिंदू देखते हैं! इससे बड़ी मूर्खता क्या होगी हिंदुओं के लिए! \nHindu Saints Targeted always\nWhy Bollywood Why https://t.co/oXNCSjTReG",https://www.twitter.com/user/status/1538119846965411840,0,"@YSSSPEAKS #PERSECUTIONOFHINDUS by Slanderers Surprisingly, every movie, every serial of Bollywood makes fun of Hindu deities!And yet Hindus see!What will be more foolish for Hindus than this!\nHindu Saints Targeted Always\nWhy bollywood why https://t.co/oxncsjtreg"
749,158291,Shalini,@Shalinisingh315,2022-06-18 02:41:28,@The_Hindu2780 BOYCOTT SUCH SERIAL SLANDERERS. \nWhy Bollywood Why \nWhy Hindu Saints Targeted ❓\nप्रकाश झा संतों के चरित्र पर बकवास करता है एक बार ये संतों के आश्रम चला जाये उसका जीवन न बदल जाये तो कहना।\nअपने चरित्र का पता नही और संतों के चरित्र पर बकवास ! \n\n#PersecutionOfHindus,https://www.twitter.com/user/status/1537988838122876928,0,"@The_Hindu2780 Boycott Such SERIAL Slanderers.\nWhy bollywood why\nWhy Hindu Saints Targeted ❓\nPrakash Jha nonsense on the character of saints, once this goes to the ashram of saints, his life does not change, then say.\nDon't know your character and fuck on the character of saints!\n\n#PERSECUTIONOFHINDUS"
770,158340,🇮🇳 𝚂𝚊𝚗𝚍𝚑𝚢𝚊 🇮🇳,@sahay_sandhya,2022-06-18 02:34:36,"@YssSpeaks #PersecutionOfHindus कर रहा है प्रकाश झा! इसने आश्रम मूवी के जरिये हमारे निर्दोष संत पर कीचड़ उछाला है, ऐसे\nSerial Slanderers \nके अपने character में कितने छेद हैं उनका नहीं पता तो संतों के चरित्र पर बकवास क्यों ? \nWhy Bollywood

In [29]:
# delete placeholders and add translated texts to originial dataframe

df = df[~df['Translated'].str.contains('Wrong')]
df = pd.concat([df, df2], axis=0, ignore_index=True)
df

,Unnamed: 0,User,Username,UTC Date,Tweet,Permalink,Is_English,Translated
0,738,Oceanman.Mp4,@Oceanman_mp4,2023-01-30 19:18:02,"@dom_vom_gym Fühlt sich ja mal giga targeted an nachdem ich heute genau deswegen beim arzt war, aber ich hab halt tatsächlich testomangel LMFAO",https://www.twitter.com/user/status/1620139293426749441,0,"@dom_vom_gym feels Giga Targeted after I was at the doctor today, but I actually have a test macker LMFAO"
1,1416,Birke,@482fj64,2023-01-30 19:00:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620134882361110529,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k"
2,1578,Annette Doerfel,@AnnetteDoerfel,2023-01-30 18:56:31,"RT @spektrum: Krebszellen gezielt anzugreifen, erhöht die Heilungschancen enorm. Mit »targeted therapies« können Ärztinnen und Ärzte bestimmte Tumoren so genau treffen, dass der Rest des Körpers weitgehend verschont bleibt. #krebs #weltkrebstag @DKFZ @XgegenKrebs https://t.co/MRNKSeyJ9K",https://www.twitter.com/user/status/1620133876755738625,0,"RT @spectrum: to target cancer cells in a targeted manner, increases the chances of recovery enormously.With ""Targeted Therapy"", doctors can hit certain tumors so precisely that the rest of the body is largely spared.#Krebs #weltkrebstag @dkfz @xGeKrebs https://t.co/mrnkseyj9k"
3,1966,VTV Gujarati News and Beyond,@VtvGujarati,2023-01-30 18:47:33,"લ્યો બોલો.. લગ્ન પ્રસંગમાં જવાનું હોવાથી દાગીના લેવા ગયાં, ઘરે ચોરે આવીને ચાર ચેઈન છૂમંતર કરી નાખી, મેઘાણીનગરનો કિસ્સો\n#Ahmedabad #VtvGujarati\n\nhttps://t.co/q0kMTF9vEy",https://www.twitter.com/user/status/1620131618588606464,0,"Speak Lyo .. Since going to the wedding, the jewelry went to pick up, the thieves came home and shot four chains, the case of Meghaninagar\n#Ahmedabad #vtvgujarati\n\nhttps://t.co/Q0KMTF9vey"
4,3000,Per-Ivar Kloen,@pikloen,2023-01-30 18:22:44,"@edelman_vincent @cbokhove @researchED_BE @researchED_NL In mijn beleving is het een genuanceerd proces. Grofweg 'targeted' voor het onthouden en geeft informatie over het leren, 'holistic' voor het verbinden van kennis en werkt interesse en motivatie in de hand.",https://www.twitter.com/user/status/1620125374389813249,0,"@edelman_vincent @cbokhove @researched_be @researched_NL In my experience it is a nuanced process.Roughly 'targeted' for remembering and gives information about learning, 'holistic' for connecting knowledge and encourages interest and motivation."
...,...,...,...,...,...,...,...,...
1990,158373,Mahima Agarwal,@MahimaA34331332,2022-06-18 02:31:23,"Why Bollywood Why \nHindu Saints Targeted\nप्रकाश झा सन्तों के चरित्र पर बकवास करता है एक बार इससे जुड़ी चरित्रहीन फीमेल्स को सन्तों के आश्रम ले जा उनका जीवन न बदल जाये तो कहना, अपने चरित्र का पता नही और सन्तों के चरित्र पर बकवास ! \nSerial Slanderers\n#PersecutionOfHindus https://t.co/MpoHB2Gjae",https://www.twitter.com/user/status/1537986302347333632,0,"Why bollywood why\nHindu Saints Targeted\nPrakash Jha nonsense on the character of saints, once the characterless female associated with it does not change their life, then say, do not know his character and nonsense on the character of saints!\nSerial Slanderers\n#PERSECUTIONOFHINDUS https://t.co/mpohb2Gjae"
1991,179372,🌷🌹Shakil Patel🌹🌷,@shakilpatel86,2022-12-15 18:47:00,Targeted Orenge gang\n\n#ShahRukhKhan \nपठान फिल्म रिकॉर्ड तोड़ेगी\n#Qatar \nकतर का विरोध किया\nफीफा विश्वकप यजमान बना\nगोबर एक ऑलराउंडर पदार्थ है\nखेत में पड़े तो खाद\nचूल्हे में पड़े तो ईंधन\nदिमाग में पड़े तो अंध भक्त बना देता है\n@ANI @BBC_WorId 